In [1]:
%%configure -f
{
"kind": "pyspark",

"conf": {
   
     "spark.master": "yarn",
    "spark.yarn.dist.archives": "/use_case/hhmatching/Library_zip/extra_lib_3.zip#lcm_zip", 
    "spark.yarn.appMasterEnv.PYSPARK_PYTHON": "./lcm_zip/bin/python",
    "spark.driver.cores": "8", 
    "spark.driver.memory": "13000m",  
    "spark.executor.memory": "13000m",
    "spark.executor.cores": "8", 
     "spark.dynamicAllocation.enabled": "true",
     "spark.dynamicAllocation.minExecutors": "3"
    
      
    }
}

### Import Libraries<a name="libs"></a>
<a href = "#toc">Back to top</a>

In [2]:
# Import required libraries
from pyspark.sql import SparkSession,SQLContext,Row, DataFrame
from pyspark import SparkContext, SparkConf, sql
import pandas as pd
import numpy as np
from pyspark.sql.types import StringType,FloatType,DoubleType,LongType,IntegerType,ArrayType
from pyspark.sql.functions import col,when,lit,abs,mean
import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp, from_unixtime,hour
from pyspark.sql.functions import explode,lower
import datetime

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def dataImport(db,table):
    df = spark.sql("select * from {0}.{1}".format(db,table))
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
mc = dataImport("sbx_adhoc_ds","spark_mob_churners_202001_202101")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
mob = dataImport("stg_dwh","stg_dwh_haloph2_mob_churn_scoring")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
mc = mc.filter(F.col("churn_period")!="CHURN_PERIOD")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
mc.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

76903

In [9]:
mob = mob.drop("churn_period","churn_reason","churn_reason_detailed","months_till_contract_end")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#fc = fc.withColumn("churn_date",F.add_months(F.col("churn_date"),0))

# fix1.withColumn("monthDiff", F.months_between(F.col("date2"), F.col("date3"))).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
mc = mc.withColumn("flag",F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
mc = mc.drop('date_of_birth')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
mob.select("calendar_period").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|calendar_period|
+---------------+
|         202006|
|         202009|
|         202101|
|         202011|
|         202007|
|         202005|
|         202012|
|         202004|
|         202102|
|         202003|
|         202010|
|         202001|
|         202008|
|         202002|
+---------------+

In [15]:
mob_all = mob.join(mc,["telephone_number_id"],"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
mob_all = mob_all.na.fill(0,["flag"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
mob_all.select("flag").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|flag|
+----+
|   1|
|   0|
+----+

In [18]:
mob_all.select([F.count(when(col(c).isNull(), c)).alias(c) for c in ["churn_period","calendar_period"]]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---------------+
|churn_period|calendar_period|
+------------+---------------+
|     9536772|              0|
+------------+---------------+

In [19]:
mob_all = mob_all.na.fill("200001",["churn_period"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
mob_all = mob_all.select(*mob_all.columns,'churn_period', from_unixtime(unix_timestamp('churn_period', 'yyyyMM')).alias('churn_date'))

mob_all= mob_all.select(*mob_all.columns, from_unixtime(unix_timestamp('calendar_period', 'yyyyMM')).alias('date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
mob_all = mob_all.withColumn("churn_date",F.add_months(F.col("churn_date"),0))
mob_all = mob_all.withColumn("date",F.add_months(F.col("date"),0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
mob_all.select("churn_date","date")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[churn_date: date, date: date]

In [23]:
# fix_all= fix_all.select(*fix_all.columns, from_unixtime(unix_timestamp('alls_period', 'yyyyMM')).alias('date'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# fix_all = fix_all.withColumn("date",F.add_months(F.col("date"),0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
mob_all1 = mob_all.withColumn("monthDiff", F.months_between(F.col("churn_date"), F.col("date")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
month1 = mob_all1.orderBy("monthDiff",ascending = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
month2 = month1.filter(F.col("flag") == 1 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
month2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

493997

In [29]:
#month3 = month2.filter((F.col("monthDiff") <= 3.0) & (F.col("monthDiff") >= 0.0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
month3 = month2.filter((F.col("monthDiff") == 4.0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
month3.select("telephone_number_id","date","churn_date","monthDiff").orderBy("monthDiff",ascending = True).show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+----------+---------+
|telephone_number_id|date      |churn_date|monthDiff|
+-------------------+----------+----------+---------+
|1189470            |2020-06-01|2020-10-01|4.0      |
|1253646            |2020-06-01|2020-10-01|4.0      |
|125592             |2020-05-01|2020-09-01|4.0      |
|1286821            |2020-08-01|2020-12-01|4.0      |
|1338421            |2020-07-01|2020-11-01|4.0      |
|1425099            |2020-01-01|2020-05-01|4.0      |
|1427365            |2020-02-01|2020-06-01|4.0      |
|1487384            |2020-08-01|2020-12-01|4.0      |
|1613416            |2020-08-01|2020-12-01|4.0      |
|1614442            |2020-03-01|2020-07-01|4.0      |
|1624220            |2020-05-01|2020-09-01|4.0      |
|1676078            |2020-04-01|2020-08-01|4.0      |
|1682929            |2020-01-01|2020-05-01|4.0      |
|1723934            |2020-09-01|2021-01-01|4.0      |
|1767698            |2020-06-01|2020-10-01|4.0      |
|1771712            |2020-04

In [32]:
month3.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48304

In [33]:
nc_month   =  month1.filter(F.col("flag") ==0 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
nc_month.select("telephone_number_id","date","churn_date","monthDiff").orderBy("monthDiff",ascending = False).show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+----------+---------+
|telephone_number_id|date      |churn_date|monthDiff|
+-------------------+----------+----------+---------+
|4525137            |2020-01-01|2000-01-01|-240.0   |
|2074563            |2020-01-01|2000-01-01|-240.0   |
|4523873            |2020-01-01|2000-01-01|-240.0   |
|1613178            |2020-01-01|2000-01-01|-240.0   |
|2072089            |2020-01-01|2000-01-01|-240.0   |
|3234999            |2020-01-01|2000-01-01|-240.0   |
|4519602            |2020-01-01|2000-01-01|-240.0   |
|1274112            |2020-01-01|2000-01-01|-240.0   |
|1613119            |2020-01-01|2000-01-01|-240.0   |
|1848673            |2020-01-01|2000-01-01|-240.0   |
|2071975            |2020-01-01|2000-01-01|-240.0   |
|2473484            |2020-01-01|2000-01-01|-240.0   |
|3234418            |2020-01-01|2000-01-01|-240.0   |
|3983630            |2020-01-01|2000-01-01|-240.0   |
|4518114            |2020-01-01|2000-01-01|-240.0   |
|1134543            |2020-01

In [35]:
nc_month.select("flag").orderBy("monthDiff",ascending = True).show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|flag|
+----+
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |
|0   |

In [36]:
nc_month.select("churn_date").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|churn_date|
+----------+
|2000-01-01|
+----------+

In [37]:
nc_month.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9536772

In [38]:
nc_month1 = nc_month.filter((F.col("date") >= F.add_months(F.current_date(), -12)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
nc_month1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7614885

In [40]:
nc_month1.select("date").distinct().show(400,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|date      |
+----------+
|2021-02-01|
|2020-11-01|
|2020-09-01|
|2021-01-01|
|2020-08-01|
|2020-05-01|
|2020-12-01|
|2020-04-01|
|2020-07-01|
|2020-10-01|
|2020-06-01|
+----------+

In [41]:
nc_month1.select("telephone_number_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

768846

In [42]:
from pyspark.sql.window import *
window_1 = Window.partitionBy(nc_month1['telephone_number_id']).orderBy(F.rand())
data_1 = nc_month1.select('*', F.rank().over(window_1).alias('rank')).filter(F.col('rank') <= 1).drop('rank')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
data_1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

768846

In [44]:
len(data_1.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

372

In [45]:
len(month3.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

372

In [46]:
mob_final = data_1.union(month3.select(*data_1.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
mob_final.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

817150

In [48]:
fix = dataImport("stg_dwh","stg_dwh_haloph2_fix_churn_scoring")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
mob_final.select("monthDiff").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|monthDiff|
+---------+
|   -244.0|
|   -245.0|
|   -247.0|
|   -248.0|
|   -252.0|
|   -246.0|
|      4.0|
|   -251.0|
|   -253.0|
|   -250.0|
|   -243.0|
|   -249.0|
+---------+

In [50]:
set(mob_final.columns) & set(fix.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'current_indicator'}

In [51]:
mob_final = mob_final.drop("current_indicator","churn_period")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:

fix= fix.select(*fix.columns, from_unixtime(unix_timestamp('alls_period', 'yyyyMM')).alias('alls_date'))

fix = fix.withColumn("alls_date",F.add_months(F.col("alls_date"),0))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
fix = fix.withColumnRenamed("oib_kupac","oib")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
L = ["MAXTV_TENURE",
"TOTAL_FAULT_DAYS_DIF_P456_P123",
"TOTAL_FAULT_DAYS_DIF_P789_P123",
"TOTAL_FAULT_DAYS_DIF_P789_P456",
"TOTAL_FAULTS_DIFF_P456_P123",
"TOTAL_FAULTS_DIFF_P789_P123",
"TOTAL_FAULTS_DIFF_P789_P456",
"CC_ALL_COUNT_LAST_3_MONTHS",
"CC_ALL_COUNT_LAST_6_MONHTS",
"CC_ALL_COUNT_LAST_9_MONTHS",
"CC_ALL_DIFF_P456_P123",
"CC_ALL_DIFF_P789_P123",
"CC_ALL_DIFF_P789_P456",
"HT_RES_SHARE_ADM_DIFF_P3_P1",
"HT_RES_SHARE_ADM_DIFF_P5_P1",
"HT_RES_SHARE_ADM_DIFF_P7_P1",
"HT_RES_SHARE_ADM_DIFF_P9_P1",
"HT_RES_SHARE_IN_ADM",
"AVG_BB_TRAFFIC_DIFF_P456_P123",
"AVG_BB_TRAFFIC_DIFF_P789_P123",
"AVG_BB_TRAFFIC_DIFF_P789_P456",
"AVG_BB_TRAFFIC_P123",
"AVG_BB_TRAFFIC_P456",
"AVG_BB_TRAFFIC_P789",
"AVG_MINUTES_DIFF_P456_P123",
"AVG_MINUTES_DIFF_P789_P123",
"AVG_MINUTES_DIFF_P789_P456",
"AVG_NR_OF_CALLS_DIFF_P456_P123",
"AVG_NR_OF_CALLS_DIFF_P789_P123",
"AVG_NR_OF_CALLS_DIFF_P789_P456"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
df2 = fix

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
df3 = df2.select("oib","alls_date",*L)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
df3 = df3.withColumn("MAXTV_TENURE",F.col("MAXTV_TENURE").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULT_DAYS_DIF_P456_P123",F.col("TOTAL_FAULT_DAYS_DIF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULT_DAYS_DIF_P789_P123",F.col("TOTAL_FAULT_DAYS_DIF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULT_DAYS_DIF_P789_P456",F.col("TOTAL_FAULT_DAYS_DIF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULTS_DIFF_P456_P123",F.col("TOTAL_FAULTS_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULTS_DIFF_P789_P123",F.col("TOTAL_FAULTS_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("TOTAL_FAULTS_DIFF_P789_P456",F.col("TOTAL_FAULTS_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_COUNT_LAST_3_MONTHS",F.col("CC_ALL_COUNT_LAST_3_MONTHS").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_COUNT_LAST_6_MONHTS",F.col("CC_ALL_COUNT_LAST_6_MONHTS").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_COUNT_LAST_9_MONTHS",F.col("CC_ALL_COUNT_LAST_9_MONTHS").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_DIFF_P456_P123",F.col("CC_ALL_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_DIFF_P789_P123",F.col("CC_ALL_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("CC_ALL_DIFF_P789_P456",F.col("CC_ALL_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P3_P1",F.col("HT_RES_SHARE_ADM_DIFF_P3_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P5_P1",F.col("HT_RES_SHARE_ADM_DIFF_P5_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P7_P1",F.col("HT_RES_SHARE_ADM_DIFF_P7_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_ADM_DIFF_P9_P1",F.col("HT_RES_SHARE_ADM_DIFF_P9_P1").cast(IntegerType()))
df3 = df3.withColumn("HT_RES_SHARE_IN_ADM",F.col("HT_RES_SHARE_IN_ADM").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_DIFF_P456_P123",F.col("AVG_BB_TRAFFIC_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_DIFF_P789_P123",F.col("AVG_BB_TRAFFIC_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_DIFF_P789_P456",F.col("AVG_BB_TRAFFIC_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_P123",F.col("AVG_BB_TRAFFIC_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_P456",F.col("AVG_BB_TRAFFIC_P456").cast(IntegerType()))
df3 = df3.withColumn("AVG_BB_TRAFFIC_P789",F.col("AVG_BB_TRAFFIC_P789").cast(IntegerType()))
df3 = df3.withColumn("AVG_MINUTES_DIFF_P456_P123",F.col("AVG_MINUTES_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_MINUTES_DIFF_P789_P123",F.col("AVG_MINUTES_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_MINUTES_DIFF_P789_P456",F.col("AVG_MINUTES_DIFF_P789_P456").cast(IntegerType()))
df3 = df3.withColumn("AVG_NR_OF_CALLS_DIFF_P456_P123",F.col("AVG_NR_OF_CALLS_DIFF_P456_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_NR_OF_CALLS_DIFF_P789_P123",F.col("AVG_NR_OF_CALLS_DIFF_P789_P123").cast(IntegerType()))
df3 = df3.withColumn("AVG_NR_OF_CALLS_DIFF_P789_P456",F.col("AVG_NR_OF_CALLS_DIFF_P789_P456").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
base1 = df3.groupBy("oib","alls_date").agg(F.max("MAXTV_TENURE").alias("MAXTV_TENURE"),
F.max("TOTAL_FAULT_DAYS_DIF_P456_P123").alias("TOTAL_FAULT_DAYS_DIF_P456_P123"),
F.max("TOTAL_FAULT_DAYS_DIF_P789_P123").alias("TOTAL_FAULT_DAYS_DIF_P789_P123"),
F.max("TOTAL_FAULT_DAYS_DIF_P789_P456").alias("TOTAL_FAULT_DAYS_DIF_P789_P456"),
F.max("TOTAL_FAULTS_DIFF_P456_P123").alias("TOTAL_FAULTS_DIFF_P456_P123"),
F.max("TOTAL_FAULTS_DIFF_P789_P123").alias("TOTAL_FAULTS_DIFF_P789_P123"),
F.max("TOTAL_FAULTS_DIFF_P789_P456").alias("TOTAL_FAULTS_DIFF_P789_P456"),
F.max("CC_ALL_COUNT_LAST_3_MONTHS").alias("CC_ALL_COUNT_LAST_3_MONTHS"),
F.max("CC_ALL_COUNT_LAST_6_MONHTS").alias("CC_ALL_COUNT_LAST_6_MONHTS"),
F.max("CC_ALL_COUNT_LAST_9_MONTHS").alias("CC_ALL_COUNT_LAST_9_MONTHS"),
F.max("CC_ALL_DIFF_P456_P123").alias("CC_ALL_DIFF_P456_P123"),
F.max("CC_ALL_DIFF_P789_P123").alias("CC_ALL_DIFF_P789_P123"),
F.max("CC_ALL_DIFF_P789_P456").alias("CC_ALL_DIFF_P789_P456"),
F.max("HT_RES_SHARE_ADM_DIFF_P3_P1").alias("HT_RES_SHARE_ADM_DIFF_P3_P1"),
F.max("HT_RES_SHARE_ADM_DIFF_P5_P1").alias("HT_RES_SHARE_ADM_DIFF_P5_P1"),
F.max("HT_RES_SHARE_ADM_DIFF_P7_P1").alias("HT_RES_SHARE_ADM_DIFF_P7_P1"),
F.max("HT_RES_SHARE_ADM_DIFF_P9_P1").alias("HT_RES_SHARE_ADM_DIFF_P9_P1"),
F.max("HT_RES_SHARE_IN_ADM").alias("HT_RES_SHARE_IN_ADM"),
F.max("AVG_BB_TRAFFIC_DIFF_P456_P123").alias("AVG_BB_TRAFFIC_DIFF_P456_P123"),
F.max("AVG_BB_TRAFFIC_DIFF_P789_P123").alias("AVG_BB_TRAFFIC_DIFF_P789_P123"),
F.max("AVG_BB_TRAFFIC_DIFF_P789_P456").alias("AVG_BB_TRAFFIC_DIFF_P789_P456"),
F.max("AVG_BB_TRAFFIC_P123").alias("AVG_BB_TRAFFIC_P123"),
F.max("AVG_BB_TRAFFIC_P456").alias("AVG_BB_TRAFFIC_P456"),
F.max("AVG_BB_TRAFFIC_P789").alias("AVG_BB_TRAFFIC_P789"),
F.max("AVG_MINUTES_DIFF_P456_P123").alias("AVG_MINUTES_DIFF_P456_P123"),
F.max("AVG_MINUTES_DIFF_P789_P123").alias("AVG_MINUTES_DIFF_P789_P123"),
F.max("AVG_MINUTES_DIFF_P789_P456").alias("AVG_MINUTES_DIFF_P789_P456"),
F.max("AVG_NR_OF_CALLS_DIFF_P456_P123").alias("AVG_NR_OF_CALLS_DIFF_P456_P123"),
F.max("AVG_NR_OF_CALLS_DIFF_P789_P123").alias("AVG_NR_OF_CALLS_DIFF_P789_P123"),
F.max("AVG_NR_OF_CALLS_DIFF_P789_P456").alias("AVG_NR_OF_CALLS_DIFF_P789_P456"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
base1.select("oib","AVG_NR_OF_CALLS_DIFF_P456_P123").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15583162

In [63]:
L1 = ["HALO_DAY_TILL_CNTRCT_EXP",
"MAXTV_DAY_TILL_CNTRCT_EXP",
"BB_SPEED_CLUSTER",
"EKVIVALENTNA_UDALJENOST",
"MAX_ADSL_ONLY_CLUSTER",
"MAX_ADSL_W_IPTV_CLUSTER",
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df4 = df2.select("oib","alls_date",*L1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
df4 = df4.withColumn("HALO_DAY_TILL_CNTRCT_EXP",F.col("HALO_DAY_TILL_CNTRCT_EXP").cast(IntegerType()))
df4 = df4.withColumn("MAXTV_DAY_TILL_CNTRCT_EXP",F.col("MAXTV_DAY_TILL_CNTRCT_EXP").cast(IntegerType()))
df4 = df4.withColumn("BB_SPEED_CLUSTER",F.col("BB_SPEED_CLUSTER").cast(IntegerType()))
df4 = df4.withColumn("EKVIVALENTNA_UDALJENOST",F.col("EKVIVALENTNA_UDALJENOST").cast(IntegerType()))
df4 = df4.withColumn("MAX_ADSL_ONLY_CLUSTER",F.col("MAX_ADSL_ONLY_CLUSTER").cast(IntegerType()))
df4 = df4.withColumn("MAX_ADSL_W_IPTV_CLUSTER",F.col("MAX_ADSL_W_IPTV_CLUSTER").cast(IntegerType()))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
base2 = df4.groupBy("oib","alls_date").agg(
F.min("HALO_DAY_TILL_CNTRCT_EXP").alias("HALO_DAY_TILL_CNTRCT_EXP"),
F.min("MAXTV_DAY_TILL_CNTRCT_EXP").alias("MAXTV_DAY_TILL_CNTRCT_EXP"),
F.min("BB_SPEED_CLUSTER").alias("BB_SPEED_CLUSTER"),
F.min("EKVIVALENTNA_UDALJENOST").alias("EKVIVALENTNA_UDALJENOST"),
F.min("MAX_ADSL_ONLY_CLUSTER").alias("MAX_ADSL_ONLY_CLUSTER"),
F.min("MAX_ADSL_W_IPTV_CLUSTER").alias("MAX_ADSL_W_IPTV_CLUSTER"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
base2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15583162

In [68]:
L2 = ["COMPLAINT_COUNT_LAST_3_MONTHS",
"COMPLAINT_COUNT_LAST_6_MONHTS",
"COMPLAINT_COUNT_LAST_9_MONTHS",
"COMPLAINT_DIFF_P456_P123",
"COMPLAINT_DIFF_P789_P123",
"COMPLAINT_DIFF_P789_P456",
"DAYS_IN_FAULT_LAST_NINE_MNTHS",
"DAYS_IN_FAULT_LAST_SIX_MNTHS",
"DAYS_IN_FAULT_LAST_THREE_MNTHS",
"FAULTS_LAST_FIX_MONHTS",
"FAULTS_LAST_NINE_MONTHS",
"FAULTS_LAST_THREE_MONTHS"]


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df5 = df2.select("oib","alls_date",*L2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
df5 = df5.withColumn("COMPLAINT_COUNT_LAST_3_MONTHS",F.col("COMPLAINT_COUNT_LAST_3_MONTHS").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_COUNT_LAST_6_MONHTS",F.col("COMPLAINT_COUNT_LAST_6_MONHTS").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_COUNT_LAST_9_MONTHS",F.col("COMPLAINT_COUNT_LAST_9_MONTHS").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_DIFF_P456_P123",F.col("COMPLAINT_DIFF_P456_P123").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_DIFF_P789_P123",F.col("COMPLAINT_DIFF_P789_P123").cast(IntegerType()))
df5 = df5.withColumn("COMPLAINT_DIFF_P789_P456",F.col("COMPLAINT_DIFF_P789_P456").cast(IntegerType()))
df5 = df5.withColumn("DAYS_IN_FAULT_LAST_NINE_MNTHS",F.col("DAYS_IN_FAULT_LAST_NINE_MNTHS").cast(IntegerType()))
df5 = df5.withColumn("DAYS_IN_FAULT_LAST_SIX_MNTHS",F.col("DAYS_IN_FAULT_LAST_SIX_MNTHS").cast(IntegerType()))
df5 = df5.withColumn("DAYS_IN_FAULT_LAST_THREE_MNTHS",F.col("DAYS_IN_FAULT_LAST_THREE_MNTHS").cast(IntegerType()))
df5 = df5.withColumn("FAULTS_LAST_FIX_MONHTS",F.col("FAULTS_LAST_FIX_MONHTS").cast(IntegerType()))
df5 = df5.withColumn("FAULTS_LAST_NINE_MONTHS",F.col("FAULTS_LAST_NINE_MONTHS").cast(IntegerType()))
df5 = df5.withColumn("FAULTS_LAST_THREE_MONTHS",F.col("FAULTS_LAST_THREE_MONTHS").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:

base3 = df5.groupBy("oib","alls_date").agg(
F.sum("COMPLAINT_COUNT_LAST_3_MONTHS").alias("COMPLAINT_COUNT_LAST_3_MONTHS"),
F.sum("COMPLAINT_COUNT_LAST_6_MONHTS").alias("COMPLAINT_COUNT_LAST_6_MONHTS"),
F.sum("COMPLAINT_COUNT_LAST_9_MONTHS").alias("COMPLAINT_COUNT_LAST_9_MONTHS"),
F.sum("COMPLAINT_DIFF_P456_P123").alias("COMPLAINT_DIFF_P456_P123"),
F.sum("COMPLAINT_DIFF_P789_P123").alias("COMPLAINT_DIFF_P789_P123"),
F.sum("COMPLAINT_DIFF_P789_P456").alias("COMPLAINT_DIFF_P789_P456"),
F.sum("DAYS_IN_FAULT_LAST_NINE_MNTHS").alias("DAYS_IN_FAULT_LAST_NINE_MNTHS"),
F.sum("DAYS_IN_FAULT_LAST_SIX_MNTHS").alias("DAYS_IN_FAULT_LAST_SIX_MNTHS"),
F.sum("DAYS_IN_FAULT_LAST_THREE_MNTHS").alias("DAYS_IN_FAULT_LAST_THREE_MNTHS"),
F.sum("FAULTS_LAST_FIX_MONHTS").alias("FAULTS_LAST_FIX_MONHTS"),
F.sum("FAULTS_LAST_NINE_MONTHS").alias("FAULTS_LAST_NINE_MONTHS"),
F.sum("FAULTS_LAST_THREE_MONTHS").alias("FAULTS_LAST_THREE_MONTHS"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
L3 = ["HH_SPEND_ON_NUM",
"RATE_HH_SUM_ARPA_M1_AVG_M23",
"RATE_HH_SUM_ARPA_M2_AVG_M31",
"RATE_HH_SUM_ARPA_M3_AVG_M12",
"RATE_HH_SUM_ARPA_M7_AVG_M89",
"RATE_HH_SUM_ARPA_M8_AVG_M97",
"RATE_HH_SUM_ARPA_M9_AVG_M78",
"RATE_SUM_ARPA_M4_AVG_M56",
"RATE_SUM_ARPA_M5_AVG_M64",
"RATE_SUM_ARPA_M6_AVG_M45",
"SUM_ARPA_HH_MIN_1",
"SUM_ARPA_HH_MIN_2",
"SUM_ARPA_HH_MIN_3",
"SUM_ARPA_HH_MIN_4",
"SUM_ARPA_HH_MIN_5",
"SUM_ARPA_HH_MIN_6",
"SUM_ARPA_HH_MIN_7",
"SUM_ARPA_HH_MIN_8",
"SUM_ARPA_HH_MIN_9",
"SUM_ARPA_TOT_HH_MIN_123",
"SUM_ARPA_TOT_HH_MIN_456",
"SUM_ARPA_TOT_HH_MIN_789",
"AVG_TOT_PAY_DIFF_P456_P123",
"AVG_TOT_PAY_DIFF_P789_P123",
"AVG_TOT_PAY_DIFF_P789_P456",
"AVG_TOTAL_PAYAMOUNT_P123",
"AVG_TOTAL_PAYAMOUNT_P456",
"AVG_TOTAL_PAYAMOUNT_P789",
"TOTAL_BILLED_FOR_LAST_3_MONTHS",
"TOTAL_BILLED_FOR_LAST_6_MONTHS",
"TOTAL_BILLED_FOR_LAST_9_MONTHS",
"AVG_MINUTES_P123",
"AVG_MINUTES_P456",
"AVG_MINUTES_P789",
"AVG_NR_OF_CALLS_P123",
"AVG_NR_OF_CALLS_P456",
"AVG_NR_OF_CALLS_P789"
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
df6 = df2.select("oib","alls_date",*L3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
df6 = df6.withColumn("HH_SPEND_ON_NUM",F.col("HH_SPEND_ON_NUM").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M1_AVG_M23",F.col("RATE_HH_SUM_ARPA_M1_AVG_M23").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M2_AVG_M31",F.col("RATE_HH_SUM_ARPA_M2_AVG_M31").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M3_AVG_M12",F.col("RATE_HH_SUM_ARPA_M3_AVG_M12").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M7_AVG_M89",F.col("RATE_HH_SUM_ARPA_M7_AVG_M89").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M8_AVG_M97",F.col("RATE_HH_SUM_ARPA_M8_AVG_M97").cast(IntegerType()))
df6 = df6.withColumn("RATE_HH_SUM_ARPA_M9_AVG_M78",F.col("RATE_HH_SUM_ARPA_M9_AVG_M78").cast(IntegerType()))
df6 = df6.withColumn("RATE_SUM_ARPA_M4_AVG_M56",F.col("RATE_SUM_ARPA_M4_AVG_M56").cast(IntegerType()))
df6 = df6.withColumn("RATE_SUM_ARPA_M5_AVG_M64",F.col("RATE_SUM_ARPA_M5_AVG_M64").cast(IntegerType()))
df6 = df6.withColumn("RATE_SUM_ARPA_M6_AVG_M45",F.col("RATE_SUM_ARPA_M6_AVG_M45").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_1",F.col("SUM_ARPA_HH_MIN_1").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_2",F.col("SUM_ARPA_HH_MIN_2").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_3",F.col("SUM_ARPA_HH_MIN_3").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_4",F.col("SUM_ARPA_HH_MIN_4").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_5",F.col("SUM_ARPA_HH_MIN_5").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_6",F.col("SUM_ARPA_HH_MIN_6").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_7",F.col("SUM_ARPA_HH_MIN_7").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_8",F.col("SUM_ARPA_HH_MIN_8").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_HH_MIN_9",F.col("SUM_ARPA_HH_MIN_9").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_TOT_HH_MIN_123",F.col("SUM_ARPA_TOT_HH_MIN_123").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_TOT_HH_MIN_456",F.col("SUM_ARPA_TOT_HH_MIN_456").cast(IntegerType()))
df6 = df6.withColumn("SUM_ARPA_TOT_HH_MIN_789",F.col("SUM_ARPA_TOT_HH_MIN_789").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOT_PAY_DIFF_P456_P123",F.col("AVG_TOT_PAY_DIFF_P456_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOT_PAY_DIFF_P789_P123",F.col("AVG_TOT_PAY_DIFF_P789_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOT_PAY_DIFF_P789_P456",F.col("AVG_TOT_PAY_DIFF_P789_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOTAL_PAYAMOUNT_P123",F.col("AVG_TOTAL_PAYAMOUNT_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOTAL_PAYAMOUNT_P456",F.col("AVG_TOTAL_PAYAMOUNT_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_TOTAL_PAYAMOUNT_P789",F.col("AVG_TOTAL_PAYAMOUNT_P789").cast(IntegerType()))
df6 = df6.withColumn("TOTAL_BILLED_FOR_LAST_3_MONTHS",F.col("TOTAL_BILLED_FOR_LAST_3_MONTHS").cast(IntegerType()))
df6 = df6.withColumn("TOTAL_BILLED_FOR_LAST_6_MONTHS",F.col("TOTAL_BILLED_FOR_LAST_6_MONTHS").cast(IntegerType()))
df6 = df6.withColumn("TOTAL_BILLED_FOR_LAST_9_MONTHS",F.col("TOTAL_BILLED_FOR_LAST_9_MONTHS").cast(IntegerType()))
df6 = df6.withColumn("AVG_MINUTES_P123",F.col("AVG_MINUTES_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_MINUTES_P456",F.col("AVG_MINUTES_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_MINUTES_P789",F.col("AVG_MINUTES_P789").cast(IntegerType()))
df6 = df6.withColumn("AVG_NR_OF_CALLS_P123",F.col("AVG_NR_OF_CALLS_P123").cast(IntegerType()))
df6 = df6.withColumn("AVG_NR_OF_CALLS_P456",F.col("AVG_NR_OF_CALLS_P456").cast(IntegerType()))
df6 = df6.withColumn("AVG_NR_OF_CALLS_P789",F.col("AVG_NR_OF_CALLS_P789").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:

base4 = df6.groupBy("oib","alls_date").agg(
F.avg("HH_SPEND_ON_NUM").alias("HH_SPEND_ON_NUM"),
F.avg("RATE_HH_SUM_ARPA_M1_AVG_M23").alias("RATE_HH_SUM_ARPA_M1_AVG_M23"),
F.avg("RATE_HH_SUM_ARPA_M2_AVG_M31").alias("RATE_HH_SUM_ARPA_M2_AVG_M31"),
F.avg("RATE_HH_SUM_ARPA_M3_AVG_M12").alias("RATE_HH_SUM_ARPA_M3_AVG_M12"),
F.avg("RATE_HH_SUM_ARPA_M7_AVG_M89").alias("RATE_HH_SUM_ARPA_M7_AVG_M89"),
F.avg("RATE_HH_SUM_ARPA_M8_AVG_M97").alias("RATE_HH_SUM_ARPA_M8_AVG_M97"),
F.avg("RATE_HH_SUM_ARPA_M9_AVG_M78").alias("RATE_HH_SUM_ARPA_M9_AVG_M78"),
F.avg("RATE_SUM_ARPA_M4_AVG_M56").alias("RATE_SUM_ARPA_M4_AVG_M56"),
F.avg("RATE_SUM_ARPA_M5_AVG_M64").alias("RATE_SUM_ARPA_M5_AVG_M64"),
F.avg("RATE_SUM_ARPA_M6_AVG_M45").alias("RATE_SUM_ARPA_M6_AVG_M45"),
F.avg("SUM_ARPA_HH_MIN_1").alias("SUM_ARPA_HH_MIN_1"),
F.avg("SUM_ARPA_HH_MIN_2").alias("SUM_ARPA_HH_MIN_2"),
F.avg("SUM_ARPA_HH_MIN_3").alias("SUM_ARPA_HH_MIN_3"),
F.avg("SUM_ARPA_HH_MIN_4").alias("SUM_ARPA_HH_MIN_4"),
F.avg("SUM_ARPA_HH_MIN_5").alias("SUM_ARPA_HH_MIN_5"),
F.avg("SUM_ARPA_HH_MIN_6").alias("SUM_ARPA_HH_MIN_6"),
F.avg("SUM_ARPA_HH_MIN_7").alias("SUM_ARPA_HH_MIN_7"),
F.avg("SUM_ARPA_HH_MIN_8").alias("SUM_ARPA_HH_MIN_8"),
F.avg("SUM_ARPA_HH_MIN_9").alias("SUM_ARPA_HH_MIN_9"),
F.avg("SUM_ARPA_TOT_HH_MIN_123").alias("SUM_ARPA_TOT_HH_MIN_123"),
F.avg("SUM_ARPA_TOT_HH_MIN_456").alias("SUM_ARPA_TOT_HH_MIN_456"),
F.avg("SUM_ARPA_TOT_HH_MIN_789").alias("SUM_ARPA_TOT_HH_MIN_789"),
F.avg("AVG_TOT_PAY_DIFF_P456_P123").alias("AVG_TOT_PAY_DIFF_P456_P123"),
F.avg("AVG_TOT_PAY_DIFF_P789_P123").alias("AVG_TOT_PAY_DIFF_P789_P123"),
F.avg("AVG_TOT_PAY_DIFF_P789_P456").alias("AVG_TOT_PAY_DIFF_P789_P456"),
F.avg("AVG_TOTAL_PAYAMOUNT_P123").alias("AVG_TOTAL_PAYAMOUNT_P123"),
F.avg("AVG_TOTAL_PAYAMOUNT_P456").alias("AVG_TOTAL_PAYAMOUNT_P456"),
F.avg("AVG_TOTAL_PAYAMOUNT_P789").alias("AVG_TOTAL_PAYAMOUNT_P789"),
F.avg("TOTAL_BILLED_FOR_LAST_3_MONTHS").alias("TOTAL_BILLED_FOR_LAST_3_MONTHS"),
F.avg("TOTAL_BILLED_FOR_LAST_6_MONTHS").alias("TOTAL_BILLED_FOR_LAST_6_MONTHS"),
F.avg("TOTAL_BILLED_FOR_LAST_9_MONTHS").alias("TOTAL_BILLED_FOR_LAST_9_MONTHS"),
F.avg("AVG_MINUTES_P123").alias("AVG_MINUTES_P123"),
F.avg("AVG_MINUTES_P456").alias("AVG_MINUTES_P456"),
F.avg("AVG_MINUTES_P789").alias("AVG_MINUTES_P789"),
F.avg("AVG_NR_OF_CALLS_P123").alias("AVG_NR_OF_CALLS_P123"),
F.avg("AVG_NR_OF_CALLS_P456").alias("AVG_NR_OF_CALLS_P456"),
F.avg("AVG_NR_OF_CALLS_P789").alias("AVG_NR_OF_CALLS_P789"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
L4 = ["TIMES_IN_CALL_CMP_LAST_3_MNTHS",
"TIMES_IN_CALL_CMP_LAST_6_MNTHS",
"TIMES_IN_CALL_CMP_LAST_9_MNTHS",
"ADM_TOT_CHURN_DIFF_P456_P123",
"ADM_TOT_CHURN_DIFF_P789_P123",
"ADM_TOT_CHURN_DIFF_P789_P456",
"ADM_TOT_CHURN_SUM_P123",
"ADM_TOT_CHURN_SUM_P456",
"ADM_TOT_CHURN_SUM_P789",
"CHA_GROUP_1",
"CHA_GROUP_2",
"CHA_GROUP_3",
"ADSL_SPEED_ACTIVE",
"ADSL_SPEED_TENURE",
"ADSL_TRAFFIC_ACTIVE",
"ADSL_TRAFFIC_TENURE",
"BB_SP_DAY_TILL_CNTRCT_EXP",
"BB_TR_DAY_TILL_CNTRCT_EXP",
"BNDL_DAY_TILL_CNTRCT_EXP",
"BUNDLE_ACTIVE",
"BUNDLE_GROUP",
"BUNDLE_TARIFF_PACKAGE",
"BUNDLE_TENURE",
"EBILL_ACTIVATED_Y_N",
"HALO_ACTIVE",
"HALO_OPTION_NON_STOP",
"HALO_TENURE",
"MAXTV_ACTIVE_MIN",
"OPIS_PONUDE",
"PLAY",
"AGE_GROUP",
"AGE_KUPAC",
"GENDER_KUPAC",
"HH_TYPE",
"HAD_INTERACTION_LAST_3_MONTHS",
"HAD_INTERACTION_LAST_6_MONTHS",
"HAD_INTERACTION_LAST_9_MONTHS",
"ACQ_MAGENTA_IN_LAST_3_MONTHS",
"ACQ_MAGENTA_IN_LAST_6_MONTHS",
"BB_TECHNOLOGY",
"FTTH_FINISHED", 
"MAGENTA_STATUS"    
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
df7 = df2.select("oib","alls_date",*L4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:

base5 = df7.groupBy("oib","alls_date").agg(
F.first("TIMES_IN_CALL_CMP_LAST_3_MNTHS").alias("TIMES_IN_CALL_CMP_LAST_3_MNTHS"),
F.first("TIMES_IN_CALL_CMP_LAST_6_MNTHS").alias("TIMES_IN_CALL_CMP_LAST_6_MNTHS"),
F.first("TIMES_IN_CALL_CMP_LAST_9_MNTHS").alias("TIMES_IN_CALL_CMP_LAST_9_MNTHS"),
F.first("ADM_TOT_CHURN_DIFF_P456_P123").alias("ADM_TOT_CHURN_DIFF_P456_P123"),
F.first("ADM_TOT_CHURN_DIFF_P789_P123").alias("ADM_TOT_CHURN_DIFF_P789_P123"),
F.first("ADM_TOT_CHURN_DIFF_P789_P456").alias("ADM_TOT_CHURN_DIFF_P789_P456"),
F.first("ADM_TOT_CHURN_SUM_P123").alias("ADM_TOT_CHURN_SUM_P123"),
F.first("ADM_TOT_CHURN_SUM_P456").alias("ADM_TOT_CHURN_SUM_P456"),
F.first("ADM_TOT_CHURN_SUM_P789").alias("ADM_TOT_CHURN_SUM_P789"),
F.first("CHA_GROUP_1").alias("CHA_GROUP_1"),
F.first("CHA_GROUP_2").alias("CHA_GROUP_2"),
F.first("CHA_GROUP_3").alias("CHA_GROUP_3"),
F.first("ADSL_SPEED_ACTIVE").alias("ADSL_SPEED_ACTIVE"),
F.first("ADSL_SPEED_TENURE").alias("ADSL_SPEED_TENURE"),
F.first("ADSL_TRAFFIC_ACTIVE").alias("ADSL_TRAFFIC_ACTIVE"),
F.first("ADSL_TRAFFIC_TENURE").alias("ADSL_TRAFFIC_TENURE"),
F.first("BB_SP_DAY_TILL_CNTRCT_EXP").alias("BB_SP_DAY_TILL_CNTRCT_EXP"),
F.first("BB_TR_DAY_TILL_CNTRCT_EXP").alias("BB_TR_DAY_TILL_CNTRCT_EXP"),
F.first("BNDL_DAY_TILL_CNTRCT_EXP").alias("BNDL_DAY_TILL_CNTRCT_EXP"),
F.first("BUNDLE_ACTIVE").alias("BUNDLE_ACTIVE"),
F.first("BUNDLE_GROUP").alias("BUNDLE_GROUP"),
F.first("BUNDLE_TARIFF_PACKAGE").alias("BUNDLE_TARIFF_PACKAGE"),
F.first("BUNDLE_TENURE").alias("BUNDLE_TENURE"),
F.first("EBILL_ACTIVATED_Y_N").alias("EBILL_ACTIVATED_Y_N"),
F.first("HALO_ACTIVE").alias("HALO_ACTIVE"),
F.first("HALO_OPTION_NON_STOP").alias("HALO_OPTION_NON_STOP"),
F.first("HALO_TENURE").alias("HALO_TENURE"),
F.first("MAXTV_ACTIVE_MIN").alias("MAXTV_ACTIVE_MIN"),
F.first("OPIS_PONUDE").alias("OPIS_PONUDE"),
F.first("PLAY").alias("PLAY"),
F.first("AGE_GROUP").alias("AGE_GROUP"),
F.first("AGE_KUPAC").alias("AGE_KUPAC"),
F.first("GENDER_KUPAC").alias("GENDER_KUPAC"),
F.first("HH_TYPE").alias("HH_TYPE"),
F.first("HAD_INTERACTION_LAST_3_MONTHS").alias("HAD_INTERACTION_LAST_3_MONTHS"),
F.first("HAD_INTERACTION_LAST_6_MONTHS").alias("HAD_INTERACTION_LAST_6_MONTHS"),
F.first("HAD_INTERACTION_LAST_9_MONTHS").alias("HAD_INTERACTION_LAST_9_MONTHS"),
F.first("ACQ_MAGENTA_IN_LAST_3_MONTHS").alias("ACQ_MAGENTA_IN_LAST_3_MONTHS"),
F.first("ACQ_MAGENTA_IN_LAST_6_MONTHS").alias("ACQ_MAGENTA_IN_LAST_6_MONTHS"),
F.first("MAGENTA_STATUS").alias("MAGENTA_STATUS"),
F.first("BB_TECHNOLOGY").alias("BB_TECHNOLOGY"),
F.first("FTTH_FINISHED").alias("FTTH_FINISHED"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
base5.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15583162

In [80]:
L5 = [
    
"HAD_COMPLAINT_LAST_3_MONTHS",
"HAD_COMPLAINT_LAST_6_MONTHS",
"HAD_COMPLAINT_LAST_9_MONTHS",
"FULL_TO_NO_MAGENTA_P2_P1",
"FULL_TO_NO_MAGENTA_P3_P2",
"FULL_TO_PARTIAL_MAGENTA_P2_P1",
"FULL_TO_PARTIAL_MAGENTA_P3_P2",
"LEFT_MAGENTA_IN_LAST_3_MONTHS",
"LEFT_MAGENTA_IN_LAST_6_MONTHS",
"PARTIAL_TO_NO_MAGENTA_P2_P1",
"PARTIAL_TO_NO_MAGENTA_P3_P2"

]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
df8 = df2.select("oib","alls_date",*L5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
def scorestr(score):
    s = "\""+score+"\""
    return s
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
quotes = F.udf(scorestr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
df8 = df8.withColumn("HAD_COMPLAINT_LAST_3_MONTHS",F.when((df8.HAD_COMPLAINT_LAST_3_MONTHS == "Y") | (df8.HAD_COMPLAINT_LAST_3_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("HAD_COMPLAINT_LAST_6_MONTHS",F.when((df8.HAD_COMPLAINT_LAST_6_MONTHS == "Y") | (df8.HAD_COMPLAINT_LAST_6_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("HAD_COMPLAINT_LAST_9_MONTHS",F.when((df8.HAD_COMPLAINT_LAST_9_MONTHS == "Y") | (df8.HAD_COMPLAINT_LAST_9_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P2_P1",F.when((df8.FULL_TO_NO_MAGENTA_P2_P1 == "Y") | (df8.FULL_TO_NO_MAGENTA_P2_P1 == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P3_P2",F.when((df8.FULL_TO_NO_MAGENTA_P3_P2 == "Y") | (df8.FULL_TO_NO_MAGENTA_P3_P2 == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P2_P1",F.when((df8.FULL_TO_PARTIAL_MAGENTA_P2_P1 == "Y") | (df8.FULL_TO_PARTIAL_MAGENTA_P2_P1 == "1"),1).otherwise(0))

df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P3_P2",F.when((df8.FULL_TO_PARTIAL_MAGENTA_P3_P2 == "Y") | (df8.FULL_TO_PARTIAL_MAGENTA_P3_P2 == "1"),1).otherwise(0))

df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_3_MONTHS",F.when((df8.LEFT_MAGENTA_IN_LAST_3_MONTHS == "Y") | (df8.LEFT_MAGENTA_IN_LAST_3_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_6_MONTHS",F.when((df8.LEFT_MAGENTA_IN_LAST_6_MONTHS == "Y") | (df8.LEFT_MAGENTA_IN_LAST_6_MONTHS == "1"),1).otherwise(0))

df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P2_P1",F.when((df8.PARTIAL_TO_NO_MAGENTA_P2_P1 == "Y") | (df8.PARTIAL_TO_NO_MAGENTA_P2_P1 == "1"),1).otherwise(0))

df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P3_P2",F.when((df8.PARTIAL_TO_NO_MAGENTA_P3_P2 == "Y") | (df8.PARTIAL_TO_NO_MAGENTA_P3_P2 == "1"),1).otherwise(0))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
# df8 = df8.withColumn("HAD_COMPLAINT_LAST_3_MONTHS",quotes("HAD_COMPLAINT_LAST_3_MONTHS"))
# df8 = df8.withColumn("HAD_COMPLAINT_LAST_6_MONTHS",quotes("HAD_COMPLAINT_LAST_6_MONTHS"))
# df8 = df8.withColumn("HAD_COMPLAINT_LAST_9_MONTHS",quotes("HAD_COMPLAINT_LAST_9_MONTHS"))
# df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P2_P1",quotes("FULL_TO_NO_MAGENTA_P2_P1"))
# df8 = df8.withColumn("FULL_TO_NO_MAGENTA_P3_P2",quotes("FULL_TO_NO_MAGENTA_P3_P2"))
# df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P2_P1",quotes("FULL_TO_PARTIAL_MAGENTA_P2_P1"))
# df8 = df8.withColumn("FULL_TO_PARTIAL_MAGENTA_P3_P2",quotes("FULL_TO_PARTIAL_MAGENTA_P3_P2"))
# df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_3_MONTHS",quotes("LEFT_MAGENTA_IN_LAST_3_MONTHS"))
# df8 = df8.withColumn("LEFT_MAGENTA_IN_LAST_6_MONTHS",quotes("LEFT_MAGENTA_IN_LAST_6_MONTHS"))
# df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P2_P1",quotes("PARTIAL_TO_NO_MAGENTA_P2_P1"))
# df8 = df8.withColumn("PARTIAL_TO_NO_MAGENTA_P3_P2",quotes("PARTIAL_TO_NO_MAGENTA_P3_P2"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
def YESorNO(data_list):
    li = np.array(data_list)
    sum1 = np.sum(li)
    if sum1 >0:
        return "Y"
    else:
        return "N"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
YESorNO = F.udf(YESorNO)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
base6 = df8.groupBy("oib","alls_date").agg(F.collect_list("HAD_COMPLAINT_LAST_3_MONTHS").alias("HAD_COMPLAINT_LAST_3_MONTHS"),F.collect_list("HAD_COMPLAINT_LAST_6_MONTHS").alias("HAD_COMPLAINT_LAST_6_MONTHS"),
F.collect_list("HAD_COMPLAINT_LAST_9_MONTHS").alias("HAD_COMPLAINT_LAST_9_MONTHS"),
F.collect_list("FULL_TO_NO_MAGENTA_P2_P1").alias("FULL_TO_NO_MAGENTA_P2_P1"),
F.collect_list("FULL_TO_NO_MAGENTA_P3_P2").alias("FULL_TO_NO_MAGENTA_P3_P2"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_P2_P1").alias("FULL_TO_PARTIAL_MAGENTA_P2_P1"),
F.collect_list("FULL_TO_PARTIAL_MAGENTA_P3_P2").alias("FULL_TO_PARTIAL_MAGENTA_P3_P2"),
F.collect_list("LEFT_MAGENTA_IN_LAST_3_MONTHS").alias("LEFT_MAGENTA_IN_LAST_3_MONTHS"),
F.collect_list("LEFT_MAGENTA_IN_LAST_6_MONTHS").alias("LEFT_MAGENTA_IN_LAST_6_MONTHS"),

F.collect_list("PARTIAL_TO_NO_MAGENTA_P2_P1").alias("PARTIAL_TO_NO_MAGENTA_P2_P1"),
F.collect_list("PARTIAL_TO_NO_MAGENTA_P3_P2").alias("PARTIAL_TO_NO_MAGENTA_P3_P2")).withColumn("HAD_COMPLAINT_LAST_3_MONTHS",YESorNO("HAD_COMPLAINT_LAST_3_MONTHS")).withColumn("HAD_COMPLAINT_LAST_6_MONTHS",YESorNO("HAD_COMPLAINT_LAST_6_MONTHS")).withColumn("HAD_COMPLAINT_LAST_9_MONTHS",YESorNO("HAD_COMPLAINT_LAST_9_MONTHS")).withColumn("FULL_TO_NO_MAGENTA_P2_P1",YESorNO("FULL_TO_NO_MAGENTA_P2_P1")).withColumn("FULL_TO_NO_MAGENTA_P3_P2",YESorNO("FULL_TO_NO_MAGENTA_P3_P2")).withColumn("FULL_TO_PARTIAL_MAGENTA_P2_P1",YESorNO("FULL_TO_PARTIAL_MAGENTA_P2_P1")).withColumn("FULL_TO_PARTIAL_MAGENTA_P3_P2",YESorNO("FULL_TO_PARTIAL_MAGENTA_P3_P2")).withColumn("LEFT_MAGENTA_IN_LAST_3_MONTHS",YESorNO("LEFT_MAGENTA_IN_LAST_3_MONTHS")).withColumn("LEFT_MAGENTA_IN_LAST_6_MONTHS",YESorNO("LEFT_MAGENTA_IN_LAST_6_MONTHS")).withColumn("PARTIAL_TO_NO_MAGENTA_P2_P1",YESorNO("PARTIAL_TO_NO_MAGENTA_P2_P1")).withColumn("PARTIAL_TO_NO_MAGENTA_P3_P2",YESorNO("PARTIAL_TO_NO_MAGENTA_P3_P2"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
base6.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15583162

In [90]:
base = base1.join(base2,["oib","alls_date"],"left")
base = base.join(base3,["oib","alls_date"],"left")
base = base.join(base4,["oib","alls_date"],"left")
base = base.join(base5,["oib","alls_date"],"left")
base = base.join(base6,["oib","alls_date"],"left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
base.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

15583162

In [107]:

# base.createOrReplaceTempView("myTemp11")
# sqlContext.sql("create table cdl_common.30march_Fix_after_agg_ as select * from myTemp11")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [98]:
Fix_mob = mob_final.join(base,(mob_final.oib == base.oib)&(mob_final.date  ==  base.alls_date),"left").drop(base.oib)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
Fix_mob.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

817150

In [99]:

Fix_mob.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.30march_mob_new_1 as select * from myTemp11")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]